In [1]:
"""Collates json-formatted results, cleans them up and saves them as .feather
files."""
# Author: William La Cava, williamlacava@gmail.com
# SRBENCH
# License: GPLv3

################################################################################
# Ground-truth problems
################################################################################
import pandas as pd
import json
import numpy as np
from glob import glob
from tqdm import tqdm
import os
import sys

rdir = '../results_sym_data/'
print('reading results from  directory', rdir)

##########
# load data from json
##########

frames = []
excluded_datasets = [
    'feynman_test_10',
    'feynman_I_26_2',
    'feynman_I_30_5'
]
excluded_cols = [
    'params'
]
fails = []
bad_bsr = []
updated = 0
for f in tqdm(glob(rdir + '/*/*.json')):
    if os.path.exists(f+'.updated'):
        f += '.updated'
        updated += 1
    if 'cv_results' in f: 
        continue
    if 'EHC' in f:
        continue
    if any([ed in f for ed in excluded_datasets]):
        continue
    try: 
        r = json.load(open(f,'r'))
        if isinstance(r['symbolic_model'],list):
            print('WARNING: list returned for model:',f)
            bad_bsr.append(f)
            sm = ['B'+str(i)+'*'+ri for i, ri in enumerate(r['symbolic_model'])]
            sm = '+'.join(sm)
            r['symbolic_model'] = sm
            
        sub_r = {k:v for k,v in r.items() if k not in excluded_cols}
    #     df = pd.DataFrame(sub_r)
        frames.append(sub_r) 
    #     print(f)
    #     print(r.keys())
    except Exception as e:
        fails.append([f,e])
        pass
    
print('{} results files loaded, {} ({:.1f}%) of which are '
	'updated'.format(len(frames), updated, updated/len(frames)*100))
print(len(fails),'fails:')
for f in fails: 
    print(f[0])
print('bad bsr:',bad_bsr)
df_results = pd.DataFrame.from_records(frames)

reading results from  directory ../results_sym_data/


100%|██████████| 2828/2828 [00:09<00:00, 311.88it/s]

2824 results files loaded, 2808 (99.4%) of which are updated
0 fails:
bad bsr: []


In [2]:
##########
# cleanup
##########

df_results = df_results.rename(columns={'time_time':'training time (s)'})
df_results.loc[:,'training time (hr)'] = df_results['training time (s)']/3600

# add modified R2 with 0 floor
df_results['r2_zero_test'] = df_results['r2_test'].apply(lambda x: max(x,0))
for col in ['symbolic_error_is_zero', 'symbolic_error_is_constant', 'symbolic_fraction_is_constant']:
    df_results.loc[:,col] = df_results[col].fillna(False)

print(','.join(df_results.algorithm.unique()))

# remove 'Regressor' from names
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('brush_C_D_UCB1','C-D-UCB1'))
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('brush_C_D_TS','C-D-TS'))
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('brush_D_UCB1','D-UCB1'))
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('brush_D_TS','D-TS'))
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('brush','Baseline'))

df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('Regressor','')) 
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('tuned.','')) 
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('sembackpropgp','SBP-GP')) 

# rename FE_AFP to AFP_FE
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('FE_AFP','AFP_FE'))

# rename GPGOMEA to GP-GOMEA
df_results['algorithm'] = df_results['algorithm'].apply(lambda x: x.replace('GPGOMEA','GP-GOMEA'))

# indicator of strogatz or feynman
df_results['data_group'] = df_results['dataset'].apply(lambda x: 'Feynman' if 'feynman' in x else 'Strogatz') 

# filling empty target noise with zeros
# df_results['target_noise']  = df_results['target_noise'].fillna(0)
# df_results['feature_noise'] = df_results['feature_noise'].fillna(0)

##########
# compute symbolic solutions
##########
print(df_results.columns)
display(df_results.sample(3).T)
df_results.loc[:,'symbolic_solution'] = df_results[['symbolic_error_is_zero',
                                                    'symbolic_error_is_constant',
                                                    'symbolic_fraction_is_constant']
                                                   ].apply(any,raw=True, axis=1)
df_results.loc[:,'symbolic_solution'] = df_results['symbolic_solution'] & ~df_results['simplified_symbolic_model'].isna() 
df_results.loc[:,'symbolic_solution'] = df_results['symbolic_solution'] & ~(df_results['simplified_symbolic_model'] == '0')
df_results.loc[:,'symbolic_solution'] = df_results['symbolic_solution'] & ~(df_results['simplified_symbolic_model'] == 'nan')

/tmp/ipykernel_93173/1602766885.py:11: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_results.loc[:,col] = df_results[col].fillna(False)


brush_C_D_UCB1,brush_D_TS,brush,brush_D_UCB1,brush_C_D_TS
Index(['dataset', 'algorithm', 'random_state', 'training time (s)',
       'symbolic_model', 'mse_train', 'mae_train', 'r2_train', 'mse_test',
       'mae_test', 'r2_test', 'simplicity', 'model_size', 'target_noise',
       'feature_noise', 'true_model', 'simplified_symbolic_model',
       'simplified_complexity', 'symbolic_error', 'symbolic_fraction',
       'symbolic_error_is_zero', 'symbolic_error_is_constant',
       'symbolic_fraction_is_constant', 'sympy_exception',
       'training time (hr)', 'r2_zero_test', 'data_group'],
      dtype='object')


,1236,1285,902
dataset,strogatz_vdp1,strogatz_vdp1,strogatz_barmag2
algorithm,D-TS,D-TS,C-D-UCB1
random_state,14423,14423,29910
training time (s),161.786293,44.804275,188.063998
symbolic_model,(sin((exp((0.47*sin((0.63*y))))-((1.33*x)--0.1...,(0.65*asin((1.54*sin((0.22*y))))),(((((-85.39-x)*(0.01*x))*sin(((-2.52*y)+-1.42)...
mse_train,0.035934,2.666984,0.0
mae_train,0.150088,0.730944,0.000461
r2_train,0.987756,0.091245,0.999995
mse_test,0.09696,3.732216,0.000001
mae_test,0.193329,0.874314,0.000445


In [3]:
print(df_results.shape)
##########
# save results
##########
df_results.to_feather('../results/ground-truth_results_local.feather')
print('results saved to ../results/ground-truth_results_local.feather')

(2824, 28)
results saved to ../results/ground-truth_results_local.feather
